# Grover's Algorithm for Finding Minimum Value in an Unsorted Array

This code is a Jupyter Notebook implementing the Grover's algorithm for searching an unsorted array of numbers. 

1. The first cell imports necessary libraries, including NumPy and Qiskit.
2. The second cell defines two functions: `find_min` and `grover_search`. The `find_min` function uses the `grover_search` function to find the minimum value in an unsorted array. The `grover_search` function implements Grover's algorithm, which is a quantum algorithm for searching an unsorted array.
3. The third cell creates an unsorted array of 16 numbers and shuffles it. It then calls the `find_min` function with this array and 64 iterations to find the minimum value.

Grover's algorithm is a quantum algorithm that can search an unsorted array in O(sqrt(N)) time, where N is the size of the array. This is much faster than classical algorithms for searching an unsorted array, which have a time complexity of O(N).

In this code, the `grover_search` function uses Qiskit's implementation of Grover's algorithm to find the index of the minimum value in the array. The function first creates an oracle circuit that marks the indices of the elements in the array that are less than or equal to a given threshold. It then applies the Grover operator to this oracle circuit, which amplifies the probability of finding the minimum value. Finally, it measures the output of the quantum circuit and returns the index of the minimum value.

Overall, this code demonstrates how to implement Grover's algorithm using Qiskit and Python.

## Versiones usadas en este cuaderno
- Python: 3.9.7 o superior (3.10.4 en entorno local)
- Quiskit 0.32 o superior (0.38.0 en entorno local)

In [3]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import GroverOperator
import random

In [6]:
# numbers = unsorted array of numbers
def find_min(numbers, iterations):
    n_items = len(numbers)
    threshold = numbers[random.randint(0, n_items-1)] # Choosing a random threshold value
    candidate_indices = [i for i in range(n_items) if numbers[i] <= threshold] # Marking elements of numbers that are <= threshold
    for _ in range(iterations):
        new_threshold = numbers[grover_search(candidate_indices, n_items)] # Grover search to find element smaller than current threshold
        if new_threshold < threshold:
            threshold = new_threshold
            candidate_indices = [idx for idx in candidate_indices if numbers[idx] <= threshold]
    return threshold

def grover_search(candidate_indices, n_items):
    n_qubits = int(np.ceil(np.log2(n_items)))
    oracle = np.identity(2**n_qubits)
    for idx in candidate_indices:
        oracle[idx, idx] = -1 # Building oracle
    oracle_circuit = QuantumCircuit(n_qubits)
    oracle_circuit.unitary(oracle, range(n_qubits))
    grover_op = GroverOperator(oracle_circuit, insert_barriers=True)
    grover_circuit = QuantumCircuit(n_qubits, n_qubits)
    grover_circuit.h(range(n_qubits)) # Initializing equal superposition
    n_grover_iterations = int(np.sqrt(n_items))
    for _ in range(n_grover_iterations):
        grover_circuit = grover_circuit.compose(grover_op)
    grover_circuit.measure(range(n_qubits), range(n_qubits))
    backend = Aer.get_backend("qasm_simulator")
    grover_circuit.decompose().draw()
    job = execute(grover_circuit, backend)
    result = int(list(job.result().get_counts().keys())[0], 2)
    print("job:", job.result().get_counts())
    return result

In [9]:
unsorted_numbers = [20, 21, 22,23,24,25,26,27,28,29,30,31,32,33,34,35]  # Example unsorted array of numbers
random.shuffle(unsorted_numbers)  # Shuffle the list to create an unsorted array
print(unsorted_numbers)
find_min(unsorted_numbers, 64)

[32, 29, 27, 23, 22, 21, 24, 31, 28, 35, 25, 20, 30, 33, 26, 34]
job: {'0001': 64, '1010': 67, '1110': 90, '0000': 93, '1000': 68, '0100': 62, '1111': 67, '0110': 74, '0011': 85, '0010': 71, '0111': 71, '1101': 65, '1100': 67, '1001': 64, '1011': 8, '0101': 8}
job: {'1100': 66, '1101': 75, '0100': 81, '0010': 73, '0111': 91, '1010': 73, '1001': 83, '1000': 67, '1111': 70, '0101': 7, '0000': 65, '0110': 66, '0011': 72, '0001': 66, '1110': 61, '1011': 8}
job: {'0011': 67, '1100': 77, '0000': 65, '1110': 74, '1101': 92, '1001': 73, '1010': 68, '0111': 69, '0010': 75, '0001': 75, '0100': 83, '1000': 54, '1111': 84, '0110': 50, '0101': 11, '1011': 7}
job: {'0011': 67, '1000': 73, '0000': 70, '1101': 73, '0100': 75, '1111': 75, '1110': 75, '1010': 60, '0010': 76, '0111': 80, '1001': 63, '0001': 71, '0110': 73, '1100': 87, '1011': 5, '0101': 1}
job: {'1001': 76, '0001': 67, '0000': 64, '1000': 79, '1011': 6, '0100': 78, '0110': 67, '1110': 80, '1100': 56, '1111': 75, '0010': 83, '1010': 84, '

20